##### *Copyright 2020 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
%env PYTHONPATH = # /env/python

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [ ]:
!conda create -n myenv python=3.6

# Retrain a detection model for Edge TPU with quant-aware training (TF 1.15)

This notebook uses a set of TensorFlow training scripts to perform transfer-learning on a quantization-aware object detection model and then convert it for compatibility with the [Edge TPU](https://coral.ai/products/).

Specifically, this tutorial shows you how to retrain a MobileNet V1 SSD model so that it detects two pets: Abyssinian cats and American Bulldogs (from the [Oxford-IIIT Pets Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)), using TensorFlow r1.15.

Beware that, compared to a desktop computer, this training can take *a lot* longer in Colab because Colab provides limited resources for long-running operations. So you'll likely see faster training speeds if you [connect this notebook to a local runtime](https://research.google.com/colaboratory/local-runtimes.html), or instead follow the [tutorial to run this training in Docker](https://coral.ai/docs/edgetpu/retrain-detection/) (which includes more documentation about this process).

## Import TensorFlow

In [1]:
%env PYTHONPATH = # /env/python

env: PYTHONPATH=# /env/python


In [2]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py38_4.12.0-Linux-x86_64.sh
!./Miniconda3-py38_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
!conda update conda

--2022-12-28 15:20:40--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76120962 (73M) [application/x-sh]
Saving to: ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’

Miniconda3-py38_4.1 100%[===================>]  72.59M   195MB/s    in 0.4s    

2022-12-28 15:20:41 (195 MB/s) - ‘Miniconda3-py38_4.12.0-Linux-x86_64.sh’ saved [76120962/76120962]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2022.3.29=h06a4308_1
    - certifi==2021.10.8=py38h06a4308_2
    - cffi==1.15.0=py38hd667e15_1
    - ch

In [3]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [4]:
!conda create -n myenv python=3.6

Solving environment: / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ | / - \ done

## Package Plan ##

  environment location: /usr/local/envs/myenv

  added / updated specs:
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py36h06a4308_0         139 KB
    pip-21.2.2                 |   py36h06a4308_0         1.8 MB
    python-3.6.13              |       h12debd9_1        32.5 MB
    setuptools-58.0.4          |   py36h06a4308_0         788 KB
    ------------------------------------------------------------
                                           Total:        35.2 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main 
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu 
  ca-certific

In [17]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

pip install tensorflow-gpu==1.15
pip install -U numpy==1.19.5
pip install -U pycocotools==2.0.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 25.0 MB/s 
     |████████████████████████████████| 499 kB 77.3 MB/s 


## Clone the model and training repos

In [6]:
! git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 80033, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 80033 (delta 98), reused 186 (delta 87), pack-reused 79825
Receiving objects: 100% (80033/80033), 594.46 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (56942/56942), done.


In [7]:
! cd models && git checkout f788046ca876a8820e05b0b48c1fc2e16b0955bc

Checking out files: 100% (4789/4789), done.
Note: checking out 'f788046ca876a8820e05b0b48c1fc2e16b0955bc'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at f788046ca Add model_dir to all tests to avoid "resource not found error". (#6143)


In [8]:
! git clone https://github.com/google-coral/tutorials.git
! cp -r tutorials/docker/object_detection/scripts/* models/research/

Cloning into 'tutorials'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 190 (delta 52), reused 58 (delta 45), pack-reused 117
Receiving objects: 100% (190/190), 128.56 KiB | 16.07 MiB/s, done.
Resolving deltas: 100% (111/111), done.


## Import dependencies

For details, see https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

In [9]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

pip install Cython contextlib2 pillow lxml jupyter matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 26.5 MB/s 
     |████████████████████████████████| 552 kB 70.5 MB/s 
     |████████████████████████████████| 121 kB 71.5 MB/s 
     |████████████████████████████████| 123 kB 65.4 MB/s 
     |████████████████████████████████| 9.9 MB 59.9 MB/s 
     |████████████████████████████████| 120 kB 61.7 MB/s 
     |████████████████████████████████| 130 kB 64.3 MB/s 
     |████████████████████████████████| 75 kB 4.7 MB/s 
     |████████████████████████████████| 427 kB 70.3 MB/s 
     |████████████████████████████████| 783 kB 58.0 MB/s 
     |████████████████████████████████| 1.4 MB 50.9 MB/s 
     |████████████████████████████████| 1.1 MB 59.8 MB/s 
     |████████████████████████████████| 386 kB 65.0 MB/s 
     |████████████████████████████████| 59 kB 7.6 MB/s 
     |████████████████████████████████| 109 kB 73.9 MB/s 
     |█████████████████████████████

In [10]:
# Get protoc 3.0.0, rather than the old version already in the container
! wget https://www.github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
! unzip protoc-3.0.0-linux-x86_64.zip -d proto3
! mkdir -p local/bin && mkdir -p local/include
! mv proto3/bin/* local/bin
! mv proto3/include/* local/include
! rm -rf proto3 protoc-3.0.0-linux-x86_64.zip

--2022-12-28 15:27:05--  https://www.github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
Resolving www.github.com (www.github.com)... 20.205.243.166
Connecting to www.github.com (www.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip [following]
--2022-12-28 15:27:05--  https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/protocolbuffers/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip [following]
--2022-12-28 15:27:06--  https://github.com/protocolbuffers/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
Reusing existing conn

In [11]:
# Install pycocoapi
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv

git clone --depth 1 https://github.com/cocodataset/cocoapi.git
(cd cocoapi/PythonAPI && make -j8)
cp -r cocoapi/PythonAPI/pycocotools/ models/research/
rm -rf cocoapi

Cloning into 'cocoapi'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 50 (delta 2), reused 19 (delta 2), pack-reused 0
Unpacking objects: 100% (50/50), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/envs/myenv/lib/python3.6/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
gcc -pthread -B /usr/local/envs/myenv/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/usr/local/envs/myen

In [12]:
# Run protoc on the object detection repo (generate .py files from .proto)
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
cd models/research/

../../local/bin/protoc object_detection/protos/*.proto --python_out=.

In [13]:
import os
os.environ['PYTHONPATH'] += ":/content/models/research:/content/models/research/slim"

Just to verify everything is correctly set up:

In [14]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
cd models/research/

python object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Running tests under Python 3.6.13: /usr/local/envs/myenv/bin/python
[ RUN      ] ModelBuilderTest.test_create_embedded_ssd_mobilenet_v1_model_from_config
[       OK ] ModelBuilderTest.test_create_embedded_ssd_mobilenet_v1_model_from_config
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_inception_resnet_v2_model_from_config
Instructions for updating:
Use `tf.cast` instead.
W1228 15:27:23.068033 140185824421760 deprecation.py:323] From /content/models/research/object_detection/anchor_generators/grid_anchor_generator.py:59: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version

## Convert training data to TFRecord

To train with different images, read [how to configure your own training data](https://coral.ai/docs/edgetpu/retrain-detection/#configure-your-own-training-data).

In [ ]:
!git clone https://github.com/joshfox10/helpers.git

In [ ]:
# Use this if data is on google drive - must mount drive first
#training folder
!cp -r "/content/drive/MyDrive/train40" "/content/train"
#validation folder
!cp -r "/content/drive/MyDrive/validation10" "/content/validation"
#test folder
!cp -r "/content/drive/MyDrive/test10" "/content/test"

In [15]:
# Use this to create folders and upload training data
!mkdir "/content/train"
!mkdir "/content/validation"
!mkdir "/content/test"

In [1]:
# Get xml_to_csv.py and generate_tfrecord.py
!git clone https://github.com/joshfox10/helpers.git

fatal: destination path 'helpers' already exists and is not an empty directory.


In [2]:
#!pip install pandas #installing pandas due to error

# %%capture
%cd /content/

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python helpers/xml_to_csv.py -i /content/train/ -o /content/train_labels.csv -l /content
# Convert test folder annotation xml files to a single csv.
!python helpers/xml_to_csv.py -i /content/validation/ -o /content/test_labels.csv

# Generate `train.record`
!python helpers/generate_tfrecord.py --csv_input=/content/train_labels.csv --output_path=/content/train.record --img_path=/content/train --label_map label_map.pbtxt
# Generate `test.record`
!python helpers/generate_tfrecord.py --csv_input=/content/test_labels.csv --output_path=/content/test.record --img_path=/content/validation --label_map label_map.pbtxt

# Set the paths
test_record_fname = '/content/test.record'
train_record_fname = '/content/train.record'
label_map_pbtxt_fname = '/content/label_map.pbtxt'

/content
Successfully converted xml to csv.
Generate `/content/label_map.pbtxt`
Successfully converted xml to csv.
Traceback (most recent call last):
  File "helpers/generate_tfrecord.py", line 22, in <module>
    import tensorflow as tf
ModuleNotFoundError: No module named 'tensorflow'
Traceback (most recent call last):
  File "helpers/generate_tfrecord.py", line 22, in <module>
    import tensorflow as tf
ModuleNotFoundError: No module named 'tensorflow'


In [ ]:
#Upload 'generate_tfrecord.py' and 'xml_to_csv.py'

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
cd models/research/

./prepare_checkpoint_and_dataset.sh --network_type mobilenet_v1_ssd --train_whole_model false

## Perform transfer-learning

The following script takes several hours to finish in Colab. (You can shorten by reducing the steps, but that reduces the final accuracy.)

If you didn't already select "Run all" then you should run all remaining cells now. That will ensure the rest of the notebook completes while you are away, avoiding the chance that the Colab runtime times-out and you lose the training data before you download the model.

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
cd models/research/

NUM_TRAINING_STEPS=500
NUM_EVAL_STEPS=100

# If you're retraining the whole model, we suggest thes values:
# %env NUM_TRAINING_STEPS=50000
# %env NUM_EVAL_STEPS=2000

./retrain_detection_model.sh --num_training_steps $NUM_TRAINING_STEPS --num_eval_steps $NUM_EVAL_STEPS

As training progresses, you can see new checkpoint files appear in the `models/research/learn_pet/train/` directory.

## Compile for the Edge TPU

In [ ]:
%%shell
eval "$(conda shell.bash hook)"
conda activate myenv
cd models/research/

NUM_TRAINING_STEPS=5

./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num $NUM_TRAINING_STEPS

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu-compiler	

In [ ]:
%cd models/research/learn_pet/models/
! ls

In [ ]:
! edgetpu_compiler output_tflite_graph.tflite

Download the files:

In [ ]:
from google.colab import files

files.download('output_tflite_graph_edgetpu.tflite')
files.download('labels.txt')

If you get a "Failed to fetch" error here, it's probably because the files weren't done saving. So just wait a moment and try again.

Also look out for a browser popup that might need approval to download the files.

## Run the model on the Edge TPU




You can now run the model on your Coral device with acceleration on the Edge TPU.

First, find some photos to try. Remember that you've trained this model to recognize just two classes: Abyssinian cats and
American Bulldogs. So here are a couple images that should provide results (provided by the
[Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html)): 

```
wget https://c4.staticflickr.com/8/7580/15865399370_ffa5b49d20_z.jpg -O dog.jpg && \
wget https://c6.staticflickr.com/9/8534/8652503705_687d957a29_z.jpg -O cat.jpg
```

Then, try running an inference using [this example code for the PyCoral API](https://github.com/google-coral/pycoral/blob/master/examples/detect_image.py). Just clone that repo and run the script using the model files you downloaded above (also be sure you have [installed the PyCoral API](https://coral.ai/software/#pycoral-api)):

```
git clone https://github.com/google-coral/pycoral

cd pycoral/examples/

python3 detect_image.py \
  --model output_tflite_graph_edgetpu.tflite \
  --labels labels.txt \
  --input dog.jpg \
  --output dog_result.jpg
```

Check out more examples for running inference at [coral.ai/examples](https://coral.ai/examples/#code-examples/).

## Implementation details



All the scripts used in this notebook come from the following locations:<br>
+  https://github.com/google-coral/tutorials/tree/master/docker/object_detection/scripts
+  https://github.com/tensorflow/models/tree/r1.13.0/research/object_detection/

More explanation of the steps in this tutorial is available at
https://coral.ai/docs/edgetpu/retrain-detection/.